# Transfer Learning com o Keras

O Keras disponibiliza alguns [modelos prontos](https://keras.io/applications/) de redes conhecidas, e alguns métodos para carregar eles sem as camadas de top/bottom.

Aqui tem um exemplo de uso de como aplicar eles com Transfer Learning para reconhecer portas e escadas utilizando a VGG 19.

No exemplo a Rede é treinada usando os ImageGenerators do Keras, que facilitam o carregamento de novas imagens, e uma rede com duas camadas fully connected com relu e um classificador softmax para reconhecer as imagens.

Vamos começar buscando o Dataset

In [2]:
import os
dataset_dir = "/media/rodsnjr/My Files/indoors/"

os.path.exists(dataset_dir)

True

# Carregar o Modelo de RNA

Para carregar o modelo basta atrelar ele numa variável, e para poder aplicar o *Transfer Learning* na rede basta setar trainable nas layers para false.

```python

for layer in model.layers:
    layer.trainable = False

```

In [3]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 224, 224

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

for layer in model.layers:
    layer.trainable = False

model.layers

Using TensorFlow backend.


# Carregar os generators

Agora criar os generators para os dados de validação, treino e testes.

E utilizar o método flow_from_directory para especificar que os dados estão vindo de um diretório.

In [4]:
train_data_dir = dataset_dir
validation_data_dir = dataset_dir

train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

Found 5418 images belonging to 3 classes.
Found 5418 images belonging to 3 classes.


# Definição da Rede de Classificação

Aqui como mencionado definimos uma rede com duas camadas Fully Connected de 1024, e com ativação relu.
E por fim uma camada de softmax para classificação.

Softmax é interessante para classificação de várias classes.

O treinamento é com SGD e Categorical Cross Entropy para essa rede.

Além disso as camadas são adicionadas na output da model definida anteriormente.

E a model completa é definida ao final, especificando que a camada de output é nosso softmax, e que a camada de entrada é a da VGG:

```python
    x = model.output
    # definições ... da rede
    predictions = (Softmax) ...
    model_final = Model(input=model.input, output = predictions)
```

In [ ]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(3, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Treinamento

Depois é só treinar o modelo.

Lembrando que o tempo de treinamento varia com o hardware, e o modelo.

A VGG 19 é um modelo relativamente ok pra treinar.
Já uma Inception precisa de um hardware melhor.

In [6]:
# Hiper Parâmetros básicos
nb_train_samples = 3000
nb_validation_samples = 1000 
batch_size = 16
epochs = 50

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

model_final.fit_generator(
train_generator,
steps_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
validation_steps = 16,
callbacks = [checkpoint, early])

Epoch 1/50


KeyboardInterrupt: 